## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точность на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [10]:
import pandas as pd
import numpy as np
import gensim
from pymorphy2 import tokenizers
import spacy
import nltk
from math import log
from sklearn import svm
import torch
from transformers import AutoTokenizer, AutoModel


In [11]:
# считываю данные
df_train = pd.read_csv('data/news_train.txt',sep='\t', header = None)
df_train.head(2)

,0,1,2
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."


## 1.

In [12]:
# функции очистки - от стоп-слов, лемматизация
from nltk.tokenize import sent_tokenize
rus_spacy = spacy.load('ru_core_news_sm')
rus_stop = set(nltk.corpus.stopwords.words('russian'))

def clean_tokens(text):
    tokens_spacy = rus_spacy(text)
    result = [str(token.lemma_).lower() for token in tokens_spacy if (str(token.lemma_).isalpha() or str(token.lemma_).isdigit()) and len(str(token.lemma_)) > 2]
    return result

In [13]:
df_train['tokens'] = df_train[2].apply(lambda x: tokenizers.simple_word_tokenize(x))
df_train['clean_tokens'] = df_train[2].apply(lambda x: clean_tokens(x))
df_train.head(1)

,0,1,2,tokens,clean_tokens
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...,"[Нападающий, «, Вашингтон, Кэпиталз, », Алекса...","[нападающий, вашингтон, кэпиталз, александр, о..."


## 2.

In [14]:
# обучение Word2Vec
model = gensim.models.Word2Vec(sentences=df_train.clean_tokens.values.tolist(), min_count=1, workers=8)

print(model['нападающий'])

[ 0.28314388 -0.28777868 -2.2720418  -0.63242996  1.3500015   0.6867338
 -0.3596796   0.9810267  -1.3352921  -0.07510228 -1.5469109   2.0361385
 -0.20431805 -1.2667714   1.3965303   0.8531669   0.06001989 -0.3522906
 -1.4320478   1.2419431  -0.62719357  1.0496353   0.36072296  1.7299687
 -2.0626934   1.0852859   0.77997047  2.580381    0.6262289  -0.39766702
 -2.4815822   0.06595197 -2.7736921   1.8702773  -0.58519566 -0.25563875
 -0.08145706 -0.7377388   0.9655309  -0.41100553  1.5970007   0.943053
 -0.1442502  -0.9689439  -0.10089649 -0.87093085 -0.97564167  0.6204158
  1.4662004   0.70554346 -0.1098365   0.4951234   0.6121644   1.5933162
 -1.0363703   0.35330403 -0.08810065 -0.8030877  -0.41920248 -1.6170603
  0.49222204 -1.1579392  -0.9281027   0.96192735 -0.22509286  1.6618536
  2.671127   -0.54887605 -0.74107426  0.42646894 -0.17938142 -2.12402
 -0.4228188  -1.2978323  -0.4956526   0.21263014 -0.36359313  0.2056176
 -0.7700504  -1.2205964   2.1848676  -0.6516354  -0.42598793 -0.4

c:\Users\Elizaveta\Classifications_2021\venv\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [15]:
# семантические ассоциации
model.wv.similar_by_word('подделка')

[('сфабрикованные', 0.8191118240356445),
 ('chan', 0.8186121582984924),
 ('краденый', 0.81550532579422),
 ('удерзо', 0.8131376504898071),
 ('подлинность', 0.8102996945381165),
 ('гудзонский', 0.8078938126564026),
 ('невиновный', 0.8052371144294739),
 ('свидетельский', 0.8042401671409607),
 ('побуждение', 0.8000742197036743),
 ('грант', 0.7975072860717773)]

In [16]:
model.wv.similar_by_word('нападающий')

[('полузащитник', 0.9831914305686951),
 ('форвард', 0.9745373725891113),
 ('голкипер', 0.9693038463592529),
 ('вратарь', 0.9632286429405212),
 ('анжи', 0.953568160533905),
 ('челси', 0.9504212737083435),
 ('реал', 0.9488823413848877),
 ('бавария', 0.9443444609642029),
 ('радулов', 0.941689670085907),
 ('ска', 0.9405420422554016)]

## 3.

In [17]:
df_test = pd.read_csv('data/news_test.txt',sep='\t', header = None)
df_test['clean_tokens'] = df_test[2].apply(lambda x: clean_tokens(x))

In [18]:
themes = list(df_train[0].unique())
all_words = list(set(model.wv.vocab.keys()))
themes_from_df = df_train[0].values.tolist()
words_from_df = df_train.clean_tokens.values.tolist()

def prob_theme(themes):
    res_list = dict()
    for theme in themes:
        res_list[theme] = 0
    kol = 0
    for all_theme in themes_from_df:
        res_list[all_theme] += 1
        kol += 1
    for i in themes:
        res_list[i] /= kol
    return res_list

def prob_words(words, themes):
    res_list = dict()
    for theme in themes:
        for word in words:
            res_list[(theme, word)] = 0
    dict_th = dict()
    for i in themes:
        dict_th[i] = 0
    for i in range(len(words_from_df)):
        for j in words_from_df[i]:
            res_list[(themes_from_df[i], j)] += 1
            dict_th[themes_from_df[i]] += 1
        
    for label, feat in res_list:
        res_list[label, feat] /= dict_th[label]
    return res_list

prob1 = prob_theme(themes)
prob2 = prob_words(all_words, themes)

In [19]:
# Байес
def klasif(sentence, classes, prob):
    return max(classes.keys(), key = lambda cl: -log(classes[cl]) + sum(-log(prob.get((cl,feat), 0) + 10**(-25), 10**(-25)) for feat in sentence))

themes_from_df = df_test[0].values.tolist()
words_from_df = df_test.clean_tokens.values.tolist()
res_test = []
for i in range(len(words_from_df)):
    res_test.append([klasif(words_from_df[i], prob1, prob2), themes_from_df[i]])
right_count = 0
for i in res_test:
    if (i[0] == i[1]):
        right_count += 1
print("Test", right_count / len(res_test))

Test 0.8033333333333333


In [20]:
# SVM
num_vocab = model.wv.vocab.copy()
for i, key in enumerate(num_vocab):
    num_vocab[key] = i
num_vocab_themes = dict()
for i, k in enumerate(themes):
    num_vocab_themes[k] = i

def to_vector(sentence):
    res = [0] * len(num_vocab)
    for i in sentence:
        if (num_vocab.get(i,0)) != 0:
            res[num_vocab[i]]+= 1
    new_res = []
    vse = 0
    summa = 0
    for i in res:
        vse += 1
        summa += i
        if (vse == 50):
            vse = 0
            new_res.append(summa)
            summa = 0
    v = np.array(new_res)
    normalized_v = v/(np.linalg.norm(v) + 0.000001)
    return normalized_v.tolist()

X_vec = []
for i in df_train.clean_tokens.values:
    X_vec.append(to_vector(i))
Y_vec = []
for i in df_train[0].values:
    Y_vec.append(num_vocab_themes[i])

my_svm = svm.SVC(kernel='rbf', gamma=5.)
my_svm.fit(X_vec, Y_vec)
my_svm.predict([X_vec[0]])

k = 0
for i in range(len(df_test[0].values)):
    if (num_vocab_themes[df_test[0].values[i]] == my_svm.predict([to_vector(df_test.clean_tokens.values[i])])[0]):
        k += 1
print('Test:', k / len(df_test[0].values))


Test: 0.77


## 4. RuBERT

In [21]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model_bert = AutoModel.from_pretrained("cointegrated/rubert-tiny")

def em_bert(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    emb = model_output.last_hidden_state[:, 0, :]
    emb = torch.nn.functional.normalize(emb)
    return emb[0].cpu().numpy()


X_train_rub = []
for i in df_train[2].values:
    X_train_rub.append(em_bert(i, model_bert, tokenizer))
Y_train_rub = []
for i in df_train[0].values:
    Y_train_rub.append(num_vocab_themes[i])

X_test_rub = []
for i in df_test[2].values:
    X_test_rub.append(em_bert(i, model_bert, tokenizer))
Y_test_rub = []
for i in df_test[0].values:
    Y_test_rub.append(num_vocab_themes[i])

svm_rub = svm.SVC(kernel='rbf', gamma=5.)
svm_rub.fit(X_train_rub, Y_train_rub)
a_rubert = svm_rub.predict([X_train_rub[0]])

k = 0
for i in range(len(X_test_rub)):
    if (Y_test_rub[i] == svm_rub.predict([X_test_rub[i]])[0]):
        k += 1
print(k / len(df_test[0].values))


Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.8393333333333334
